# Classification Dataset

### Data Mining Project 2024/25

Authors: Nicola Emmolo, Simone Marzeddu, Jacopo Raffi

In [28]:
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.over_sampling import ADASYN
from scipy.stats import ks_2samp

import pandas as pd
import numpy 
from sklearn.model_selection import train_test_split
import seaborn as sb

In [29]:
dataset = pd.read_csv('../../data/complete_dataset_fe.csv', dtype={'cyclist_team': str})
RANDOM_STATE = 42   

In [30]:
dataset['label'] = (dataset['raw_position'] <= 20).astype(int) #1 if top 20, 0 otherwise 

In [31]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589739 entries, 0 to 589738
Data columns (total 38 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   cyclist_url                  589739 non-null  object 
 1   cyclist_name                 589739 non-null  object 
 2   birth_year                   589739 non-null  int64  
 3   weight                       480007 non-null  float64
 4   height                       480007 non-null  float64
 5   nationality                  589739 non-null  object 
 6   race_url                     589739 non-null  object 
 7   race_name                    589739 non-null  object 
 8   points                       589739 non-null  float64
 9   uci_points                   422184 non-null  float64
 10  length                       589739 non-null  float64
 11  climb_total                  475338 non-null  float64
 12  profile                      475338 non-null  float64
 13 

Feature da eliminare:

- cyclist_url (non generalizzabili)
- cyclist_name (non generalizzabili)
- birth_year (gia incluso in age)
- nationality (troppo sbilanciamento nel dataset)
- weight, height (gia incluso in BMI)
- uci_points (troppo pochi e gia inclusi in points)
- race_url (non generalizzabili)
- race_name (non generalizzabili)
- climb_total, profile (gia inclusi in race_physical_effort)
- mostly_tarmac (sbilanciato)
- cyclist_team (non generalizzabili)
- date (non generalizzabili)
- raw_position, position (obv)
- delta (obv) 
- race_year (non generalizzabili)
- race_stage (non generalizzabili)
- cyclist_age_group (??)
- cyclist_climb_power (feature generata con delta, obv)
- points, startlist_quality (gia inclusti in race_prestige)

We must choose between cyclist_age and cyclist_age_group as both these features represent the same information bu give different advantage/disadvantage to our classification process: 
- cyclist_age -> lower dimensionality (no one-hot) but unbalanced representation
- cyclist_age_group -> arbitraty encoding but balanced representation 

we chose to delete cyclist_age as the encoding is arbitraty but it still respects the "order" of the age_groups

In [32]:
columns = ['length', 'race_season', 'cyclist_bmi', 'cyclist_age_group', 'climb_percentage', 'race_physical_effort', 'race_prestige', 'race_year', 'label']
dataset = dataset[columns].dropna()

In [33]:
development_dataset = dataset[dataset['race_year'] < 2022]
test_dataset = dataset[dataset['race_year'] >= 2022]

development_dataset = development_dataset.drop(columns=['race_year'])
test_set = test_dataset.drop(columns=['race_year'])
test_label = test_set.pop('label')

In [34]:
label = development_dataset.pop('label')
train_set, val_set, train_label, val_label = train_test_split(development_dataset, label, stratify=label, test_size=0.20, random_state=RANDOM_STATE)

In [35]:
development_set = development_dataset.copy()
developmente_label = label

In [36]:
developmente_label.value_counts(normalize=True)

label
0    0.854316
1    0.145684
Name: proportion, dtype: float64

In [37]:
development_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 367631 entries, 4 to 589737
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   length                367631 non-null  float64
 1   race_season           367631 non-null  object 
 2   cyclist_bmi           367631 non-null  float64
 3   cyclist_age_group     367631 non-null  object 
 4   climb_percentage      367631 non-null  float64
 5   race_physical_effort  367631 non-null  float64
 6   race_prestige         367631 non-null  float64
dtypes: float64(5), object(2)
memory usage: 22.4+ MB


In [38]:
def discretize_data(dataset, variables):
    for variable in variables:
        #get the unique variable's values
        var = sorted(dataset[variable].unique())
        
        #generate a mapping from the variable's values to the number representation  
        mapping = dict(zip(var, range(0, len(var) + 1)))

        #add a new colum with the number representation of the variable
        dataset[variable+'_num'] = dataset[variable].map(mapping).astype(int)
    return dataset

In [ ]:
variables = ['cyclist_age_group']
development_set = discretize_data(development_set, variables)
development_set = development_set.drop(columns=variables)

In [ ]:
categorical_features = dataset.select_dtypes(exclude="number").columns.tolist()
models = [
    RandomOverSampler(random_state=RANDOM_STATE),
    SMOTE(random_state=RANDOM_STATE, k_neighbors=10),
    SMOTENC(random_state=RANDOM_STATE, k_neighbors=10, categorical_features=categorical_features),
    ADASYN(random_state=RANDOM_STATE, n_neighbors=10)
]
oversampling_algorithms = [
    "random",
    "smote_interpolation",
    "smote_interpolation_w_categorical",
    "adasyn"
]
oversampled_datasets = list()

for algorithm, model in zip(oversampling_algorithms, models):
    if algorithm in ("smote_interpolation", "adasyn"):
        oversampled_data, oversampled_labels = model.fit_resample(development_set.select_dtypes(include="number"), developmente_label)
        oversampled_dataset = pd.DataFrame(numpy.hstack((oversampled_data, oversampled_labels)), columns=dataset.select_dtypes(include="number").columns)
    else:
        oversampled_data, oversampled_labels = model.fit_resample(development_set, developmente_label)
        oversampled_dataset = pd.DataFrame(numpy.hstack((oversampled_data, oversampled_labels)), columns=dataset.columns)
    oversampled_dataset["algorithm"] = algorithm

    oversampled_datasets.append(oversampled_dataset)

In [ ]:
tests_per_algorithm = list()
for algorithm, oversampled_dataset in zip(oversampling_algorithms, oversampled_datasets):
    columns = oversampled_dataset.columns
    test_results = [
        ks_2samp(
            dataset[column],
            oversampled_dataset[column],
            alternative="two-sided"
        )
        for column in columns if column != "algorithm"
    ]
    test_data = [(
        test.statistic,
        test.pvalue,
        test.statistic_location
        )
        for test in test_results        
    ]
    test_data = pd.DataFrame(test_data, columns=["KS_test", "p_value", "margin"])
    test_data["algorithm"] = algorithm

    tests_per_algorithm.append(test_data)

validation = pd.concat(tests_per_algorithm, axis="rows")
validation.groupby("algorithm").describe()

,KS_test,p_value
count,8.000000,8.000000
mean,0.049204,0.361460
std,0.123402,0.389387
min,0.001895,0.000000
25%,0.002695,0.015146
50%,0.003967,0.224555
75%,0.008941,0.691544
max,0.354316,0.963775


In [ ]:
dev_oversample = oversampled_dataset #TODO: choose the best one
dev_oversample['label'].value_counts(normalize=True)

label
0    0.5
1    0.5
Name: proportion, dtype: float64

In [ ]:
dev_label = dev_oversample.pop('label')

In [ ]:
#drop the categorical variables since we don't need them anymore 
dev_oversample.drop(columns=variables, axis=1,inplace=True)
cat_columns = ['race_season']
dev_oversample_encoded = pd.get_dummies(dev_oversample, columns = cat_columns, prefix_sep='%')
dev_oversample_encoded

,length,cyclist_bmi,climb_percentage,race_physical_effort,race_prestige,cyclist_age_group_num,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
0,202.0,19.789698,0.01697,0.234084,0.102308,1,False,True,False,False
1,207.0,20.830818,0.0177,0.170928,0.122411,1,False,False,True,False
2,150.3,18.376825,0.023287,0.237108,0.068472,0,False,False,False,True
3,7.0,20.897959,0.029571,0.000162,0.161833,0,False,False,True,False
4,24.0,19.835163,0.023375,0.001516,0.069833,2,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
85687,158.7,20.047446,0.016364,0.139324,0.137776,4,False,False,True,False
85688,209.6,20.047446,0.010215,0.1517,0.137776,4,False,False,True,False
85689,200.3,20.047446,0.014518,0.196904,0.137776,4,False,False,True,False
85690,173.0,20.047446,0.012566,0.08476,0.137776,4,False,False,True,False


In [21]:
test_set = discretize_data(test_set, variables)
test_set.drop(columns=variables, axis=1,inplace=True)
cat_columns = ['race_season']
test_set_encoded = pd.get_dummies(test_set, columns = cat_columns, prefix_sep='%')
test_set_encoded

,length,cyclist_bmi,climb_percentage,race_physical_effort,race_prestige,cyclist_age_group_num,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
201,203.9,21.705129,0.009858,0.092363,0.225487,3,False,False,False,True
202,254.1,21.705129,0.013617,0.198139,0.248896,3,False,True,False,False
204,204.2,20.756387,0.008159,0.076668,0.264741,0,False,False,False,True
205,203.9,20.756387,0.009858,0.092363,0.225487,0,False,False,False,True
206,183.7,20.756387,0.008977,0.068267,0.228368,0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...
589373,154.0,20.305175,0.010779,0.028805,0.078556,4,False,False,False,True
589374,294.0,20.305175,0.007327,0.142718,0.455848,4,False,False,False,True
589375,204.1,20.305175,0.009882,0.092776,0.304715,4,False,False,False,True
589376,273.4,20.305175,0.007992,0.134628,0.399068,4,False,True,False,False


Abbiamo trasformato gli attributi categorici in equivalenti numerici per poter essere processati dai modelli di learning: 
- one-hot encoding: è stato utilizzato quando l'attributo categorico non è di tipo ordinale, in questo caso non siamo interessati a preservare ordine o relazioni di vicinanza tra i valori di questo attributo e pertanto vogliamo massimizzare la dissimilarità tra le classi
- discretizzazione: abbiamo usato questa tecnica solo nel caso dell'attributo cyclist_age_group, in quanto queste classi possiedono relazioni di ordine e vicinanza (attributo ordinale)

In [ ]:
dev_oversample_encoded['label'] = dev_label
test_set_encoded['label'] = test_label

In [ ]:
#dev_oversample_encoded.to_csv('../data/ml_datasets/oversampling/dev_set.csv', index=False)
#test_set_encoded.to_csv('../data/ml_datasets/oversampling/test_set.csv', index=False)